# A program For Data Science Conference

Introduction<br>
AutoTrader is a company based in Manchester is organizing event for 5 days for a group of Data Scientists from all over the world.The company has to put a good program, including a hotel of residence, a hall for meetings, places of landscape to visit, stores for shopping, restaurants,carparks and cafes. So the company’s purpose is to make a list of places of landscape in Manchester, including the nearest restaurants, cafes, and shopping stores for each place.<br>
<hr>
Data Description<br>
The data used in this project is provided by Foursquare location data. The data are grouped by landscape area, and each area included the information about this area and all information about restaurants, cafes, and stores which in this area.
<hr>
Table of contents<br>
1. Import Libraries<br>
2. Define Foursquare Credentials<br>
3. Define the city and get its latitude & longitude<br>
4. Search for Hotels & clean dataframe<br>
5. Search for Parking & clean dataframe<br>
6. Search for Restaurants & clean dataframe<br>
7. Search for Shopping Stores & clean dataframe<br>
8. Generate map to visualize hotels, shopping stores,Parking and Restaurant and how they cluster together<br>
<hr>

## Import Libraries

In [1]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('packages imported')


packages imported


## Define Foursquare Credentials

In [2]:
# @hidden_cell
ClIENT_ID = 'A4WVHVVXOXM5QXI4I4BYJQEDSSOSZSVR5SDROS5UVZCONL0O' # your Foursquare ID
ClIENT_SECRET = '1B0N25JWRZLFAS4EIBSK4XSCAAF2H3Z0LQCDAF5PM3YBIKQA' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30

## Define the city and get its latitude & longitude

In [3]:
# define the city and get its latitude & longitude 
city = 'Manchester'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

53.4794892 -2.2451148


## Hotel Search

In [4]:
# Search hotels and get hotel dataset. 
search_query = 'Hotel'
radius = 700

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

# Send the GET Request and examine the results
results = requests.get(url).json()

venues = results['response']['venues']

# tranform venues into a dataframe
data = json_normalize(venues)
data.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ade0dd3f964a520456d21e3,Portland Street,GB,Manchester,United Kingdom,NaN,511,"[Portland Street, Manchester, M1 4PH, United K...","[{'label': 'display', 'lat': 53.47963831318294...",53.479638,-2.237399,NaN,M1 4PH,NaN,Mercure Manchester Piccadilly Hotel,v-1576841980,NaN
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ade0dd9f964a520696d21e3,Peter Street,GB,Manchester,United Kingdom,NaN,220,"[Peter Street, Manchester, Greater Manchester,...","[{'label': 'display', 'lat': 53.47750796558518...",53.477508,-2.244989,NaN,M60 2DS,Greater Manchester,The Midland Hotel,v-1576841980,90978518
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ade0ddaf964a5206f6d21e3,18-24 Princess Street,GB,Manchester,United Kingdom,NaN,267,"[18-24 Princess Street, Manchester, Greater Ma...","[{'label': 'display', 'lat': 53.47824112955562...",53.478241,-2.241658,NaN,M1 4LY,Greater Manchester,Princess St. Hotel,v-1576841980,NaN
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ade0dd2f964a520436d21e3,Portland St,GB,Manchester,United Kingdom,NaN,523,"[Portland St, Manchester, Greater Manchester, ...","[{'label': 'display', 'lat': 53.47905633448154...",53.479056,-2.237239,NaN,M1 3LA,Greater Manchester,Britannia Hotel Manchester,v-1576841980,NaN
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ade0dd3f964a520486d21e3,Blackfriars St,GB,Manchester,United Kingdom,NaN,510,"[Blackfriars St, Manchester, Greater Mancheste...","[{'label': 'display', 'lat': 53.48401201209956...",53.484012,-2.246379,NaN,M3 2EQ,Greater Manchester,Renaissance Manchester City Centre Hotel,v-1576841980,71468424


In [5]:
# Data cleaning
clean_columns = ['name', 'categories'] + [col for col in data.columns if col.startswith('location.')]+ ['id']
clean_data = data.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_data['categories'] = clean_data.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_data.columns = [column.split('.')[-1] for column in clean_data.columns]

clean_data.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Mercure Manchester Piccadilly Hotel,Hotel,Portland Street,GB,Manchester,United Kingdom,NaN,511,"[Portland Street, Manchester, M1 4PH, United K...","[{'label': 'display', 'lat': 53.47963831318294...",53.479638,-2.237399,NaN,M1 4PH,NaN,4ade0dd3f964a520456d21e3
1,The Midland Hotel,Hotel,Peter Street,GB,Manchester,United Kingdom,NaN,220,"[Peter Street, Manchester, Greater Manchester,...","[{'label': 'display', 'lat': 53.47750796558518...",53.477508,-2.244989,NaN,M60 2DS,Greater Manchester,4ade0dd9f964a520696d21e3
2,Princess St. Hotel,Hotel,18-24 Princess Street,GB,Manchester,United Kingdom,NaN,267,"[18-24 Princess Street, Manchester, Greater Ma...","[{'label': 'display', 'lat': 53.47824112955562...",53.478241,-2.241658,NaN,M1 4LY,Greater Manchester,4ade0ddaf964a5206f6d21e3
3,Britannia Hotel Manchester,Hotel,Portland St,GB,Manchester,United Kingdom,NaN,523,"[Portland St, Manchester, Greater Manchester, ...","[{'label': 'display', 'lat': 53.47905633448154...",53.479056,-2.237239,NaN,M1 3LA,Greater Manchester,4ade0dd2f964a520436d21e3
4,Renaissance Manchester City Centre Hotel,Hotel,Blackfriars St,GB,Manchester,United Kingdom,NaN,510,"[Blackfriars St, Manchester, Greater Mancheste...","[{'label': 'display', 'lat': 53.48401201209956...",53.484012,-2.246379,NaN,M3 2EQ,Greater Manchester,4ade0dd3f964a520486d21e3


In [6]:
# delete unnecessary columns
clean_data= clean_data.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)

clean_data.head()

,name,categories,address,lat,lng,postalCode,state
0,Mercure Manchester Piccadilly Hotel,Hotel,Portland Street,53.479638,-2.237399,M1 4PH,NaN
1,The Midland Hotel,Hotel,Peter Street,53.477508,-2.244989,M60 2DS,Greater Manchester
2,Princess St. Hotel,Hotel,18-24 Princess Street,53.478241,-2.241658,M1 4LY,Greater Manchester
3,Britannia Hotel Manchester,Hotel,Portland St,53.479056,-2.237239,M1 3LA,Greater Manchester
4,Renaissance Manchester City Centre Hotel,Hotel,Blackfriars St,53.484012,-2.246379,M3 2EQ,Greater Manchester


In [7]:
# delete rows with none values (missing values)
clean_data = clean_data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_data.head()

,name,categories,address,lat,lng,postalCode,state
1,The Midland Hotel,Hotel,Peter Street,53.477508,-2.244989,M60 2DS,Greater Manchester
2,Princess St. Hotel,Hotel,18-24 Princess Street,53.478241,-2.241658,M1 4LY,Greater Manchester
3,Britannia Hotel Manchester,Hotel,Portland St,53.479056,-2.237239,M1 3LA,Greater Manchester
4,Renaissance Manchester City Centre Hotel,Hotel,Blackfriars St,53.484012,-2.246379,M3 2EQ,Greater Manchester
5,Manchester Marriott Victoria & Albert Hotel,Hotel,Water Street,53.479565,-2.256742,M3 4JQ,Greater Manchester


In [8]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_data= clean_data.loc[clean_data['categories'].isin(array)]
hotel_data.head()

,name,categories,address,lat,lng,postalCode,state
1,The Midland Hotel,Hotel,Peter Street,53.477508,-2.244989,M60 2DS,Greater Manchester
2,Princess St. Hotel,Hotel,18-24 Princess Street,53.478241,-2.241658,M1 4LY,Greater Manchester
3,Britannia Hotel Manchester,Hotel,Portland St,53.479056,-2.237239,M1 3LA,Greater Manchester
4,Renaissance Manchester City Centre Hotel,Hotel,Blackfriars St,53.484012,-2.246379,M3 2EQ,Greater Manchester
5,Manchester Marriott Victoria & Albert Hotel,Hotel,Water Street,53.479565,-2.256742,M3 4JQ,Greater Manchester


In [9]:
# delete rows which has duplicate hotel's name
print('Hotel data size:',hotel_data.shape)
hotel_data = hotel_data.drop_duplicates(subset='name', keep="first")
print('Hotel data size:',hotel_data.shape)

Hotel data size: (9, 7)
Hotel data size: (9, 7)


In [10]:
# choose the hotel which has the same postalCode with the event space
df_hotel = hotel_data[hotel_data.postalCode == 'M1 3LA']
df_hotel

,name,categories,address,lat,lng,postalCode,state
3,Britannia Hotel Manchester,Hotel,Portland St,53.479056,-2.237239,M1 3LA,Greater Manchester


## Parking Search

In [11]:
# search for Parks
search_query = 'Park'
radius = 100000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

# Send the GET Request and examine the results
park_results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = park_results['response']['venues']

# tranform venues into a dataframe
park_data = json_normalize(venues)
park_data.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4b5df776f964a520867629e3,4 Cheetham Hill Road,GB,Manchester,United Kingdom,NaN,1189,"[4 Cheetham Hill Road, Manchester, M4 4EW, Uni...","[{'label': 'display', 'lat': 53.48993409245847...",53.489934,-2.241320,M4 4EW,NaN,Park Inn by Radisson,v-1576841980
1,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'P...",False,5a673109a22db74fd8af0a6b,Major St,GB,Manchester,United Kingdom,NaN,638,"[Major St, Manchester, Greater Manchester, M1,...","[{'label': 'display', 'lat': 53.479008, 'lng':...",53.479008,-2.235512,M1,Greater Manchester,NCP Major Street Car Park,v-1576841980
2,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'P...",False,4f3cdccaa17c8a4ed1dbf1fe,75 St James Street,GB,Manchester,United Kingdom,NaN,325,"[75 St James Street, Manchester, Greater Manch...","[{'label': 'display', 'lat': 53.477661, 'lng':...",53.477661,-2.241282,M1 4BP,Greater Manchester,"Q-Park Piazza Car Park, Manchester",v-1576841980
3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4c1cdccbb306c9288e7c64b7,NaN,GB,Manchester,United Kingdom,NaN,1212,"[Manchester, United Kingdom]","[{'label': 'display', 'lat': 53.46940145799939...",53.469401,-2.252026,NaN,NaN,Hulme Park,v-1576841980
4,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'P...",False,5676ca6b498e97455c048894,2 Chapel Street,GB,Manchester,United Kingdom,NaN,715,"[2 Chapel Street, Manchester, Greater Manchest...","[{'label': 'display', 'lat': 53.485912, 'lng':...",53.485912,-2.245509,M3 7WJ,Greater Manchester,"Q-Park Deansgate North Car Park, Manchester",v-1576841980


In [12]:
# park data cleaning

# keep only columns that include venue name, and anything that is associated with location
park_clean_columns = ['name', 'categories'] + [col for col in park_data.columns if col.startswith('location.')]+ ['id']
clean_park_data = park_data.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_data['categories'] = clean_park_data.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_data.columns = [column.split('.')[-1] for column in clean_park_data.columns]

clean_park_data.head()


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Park Inn by Radisson,Hotel,4 Cheetham Hill Road,GB,Manchester,United Kingdom,NaN,1189,"[4 Cheetham Hill Road, Manchester, M4 4EW, Uni...","[{'label': 'display', 'lat': 53.48993409245847...",53.489934,-2.241320,M4 4EW,NaN,4b5df776f964a520867629e3
1,NCP Major Street Car Park,Parking,Major St,GB,Manchester,United Kingdom,NaN,638,"[Major St, Manchester, Greater Manchester, M1,...","[{'label': 'display', 'lat': 53.479008, 'lng':...",53.479008,-2.235512,M1,Greater Manchester,5a673109a22db74fd8af0a6b
2,"Q-Park Piazza Car Park, Manchester",Parking,75 St James Street,GB,Manchester,United Kingdom,NaN,325,"[75 St James Street, Manchester, Greater Manch...","[{'label': 'display', 'lat': 53.477661, 'lng':...",53.477661,-2.241282,M1 4BP,Greater Manchester,4f3cdccaa17c8a4ed1dbf1fe
3,Hulme Park,Park,NaN,GB,Manchester,United Kingdom,NaN,1212,"[Manchester, United Kingdom]","[{'label': 'display', 'lat': 53.46940145799939...",53.469401,-2.252026,NaN,NaN,4c1cdccbb306c9288e7c64b7
4,"Q-Park Deansgate North Car Park, Manchester",Parking,2 Chapel Street,GB,Manchester,United Kingdom,NaN,715,"[2 Chapel Street, Manchester, Greater Manchest...","[{'label': 'display', 'lat': 53.485912, 'lng':...",53.485912,-2.245509,M3 7WJ,Greater Manchester,5676ca6b498e97455c048894


In [13]:
# delete unnecessary columns
clean_park_data= clean_park_data.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)

clean_park_data.head()

,name,categories,address,lat,lng,postalCode,state
0,Park Inn by Radisson,Hotel,4 Cheetham Hill Road,53.489934,-2.241320,M4 4EW,NaN
1,NCP Major Street Car Park,Parking,Major St,53.479008,-2.235512,M1,Greater Manchester
2,"Q-Park Piazza Car Park, Manchester",Parking,75 St James Street,53.477661,-2.241282,M1 4BP,Greater Manchester
3,Hulme Park,Park,NaN,53.469401,-2.252026,NaN,NaN
4,"Q-Park Deansgate North Car Park, Manchester",Parking,2 Chapel Street,53.485912,-2.245509,M3 7WJ,Greater Manchester


In [14]:
# delete rows with none values (missing values)
clean_park_data = clean_park_data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_data.head()

,name,categories,address,lat,lng,postalCode,state
1,NCP Major Street Car Park,Parking,Major St,53.479008,-2.235512,M1,Greater Manchester
2,"Q-Park Piazza Car Park, Manchester",Parking,75 St James Street,53.477661,-2.241282,M1 4BP,Greater Manchester
4,"Q-Park Deansgate North Car Park, Manchester",Parking,2 Chapel Street,53.485912,-2.245509,M3 7WJ,Greater Manchester
6,"Q-Park Piccadilly Place Car Park, Manchester",Parking,Whitworth Street,53.475538,-2.239354,M1 3BN,Greater Manchester
7,Manchester Fort Retail Park,Shopping Plaza,Cheetham Hill Road,53.497997,-2.235790,M8 8EP,Greater Manchester


In [15]:
# carparks near event 
clean_park_data[(clean_park_data.postalCode.str.contains('M1 ')) &(clean_park_data.categories=='Parking')]

,name,categories,address,lat,lng,postalCode,state
2,"Q-Park Piazza Car Park, Manchester",Parking,75 St James Street,53.477661,-2.241282,M1 4BP,Greater Manchester
6,"Q-Park Piccadilly Place Car Park, Manchester",Parking,Whitworth Street,53.475538,-2.239354,M1 3BN,Greater Manchester


## Search for Restaurants 

In [16]:
# search for Restaurants
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

Restaurant_data = requests.get(url).json()

# assign relevant part of JSON to venues
venues = Restaurant_data['response']['venues']

# tranform venues into a dataframe
Restaurant_data = json_normalize(venues)

# data cleaning 
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_data.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_data = Restaurant_data.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_data['categories'] = clean_Restaurant_data.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_data.columns = [column.split('.')[-1] for column in clean_Restaurant_data.columns]

# delete unnecessary columns
clean_Restaurant_data= clean_Restaurant_data.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)

# delete rows with none values
df_Restaurant = clean_Restaurant_data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant.head()

,name,categories,address,lat,lng,postalCode,state
1,The Restaurant Bar and Grill,Steakhouse,14 John Dalton Street,53.480527,-2.247170,M2 6JR,Greater Manchester
2,Gino D'Acampo - My Restaurant,Italian Restaurant,98 Corporation St,53.484844,-2.242675,M4 3TR,Greater Manchester
9,Tae's Tavern - Caribbean Restaurant Bar Cafe,Caribbean Restaurant,187 - 189 Chapel Street,53.483456,-2.255137,M3 5EQ,Greater Manchester
10,The Glasshouse Bar & Restaurant,Bar,70 Shudehill,53.485774,-2.236463,M4 4AF,Greater Manchester
13,Wings Restaurant,Chinese Restaurant,Brazenose St.,53.479729,-2.247190,M2 5LN,Greater Manchester


## Search for Shopping Stores

In [17]:
# search for Shopping
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

# Send the GET Request and examine the results
sresults = requests.get(url).json()

# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)

# clean shopping data 
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]
clean_Shopping_dataframe.head()


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Manchester Arndale,Shopping Mall,High St,GB,Manchester,United Kingdom,NaN,541,"[High St, Manchester, M4 3AQ, United Kingdom]","[{'label': 'display', 'lat': 53.48378988491711...",53.483790,-2.241297,M4 3AQ,NaN,4ade0e37f964a5206a6f21e3
1,Royal Exchange Arcade,Shopping Mall,St Ann's Square,GB,Manchester,United Kingdom,NaN,355,"[St Ann's Square, Manchester, M2 7EA, United K...","[{'label': 'display', 'lat': 53.48263760357169...",53.482638,-2.244235,M2 7EA,NaN,4ade0e3df964a5208f6f21e3
2,Sports Direct,Sporting Goods Shop,"Unit MSU2, Arndale Shopping Centre",GB,Manchester,United Kingdom,Lvl 18,589,"[Unit MSU2, Arndale Shopping Centre (Lvl 18), ...","[{'label': 'display', 'lat': 53.48377034900374...",53.483770,-2.239871,M4 2HU,Greater Manchester,4c111c1eb93cc9b62ecf57e0


In [18]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)

# delete rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping.head()

,name,categories,address,lat,lng,postalCode,state
0,Manchester Arndale,Shopping Mall,High St,53.483790,-2.241297,M4 3AQ,NaN
1,Royal Exchange Arcade,Shopping Mall,St Ann's Square,53.482638,-2.244235,M2 7EA,NaN


## Generate maps to visualize venues and how they cluster together

In [19]:
# create dataframe of hotels, shopping stores and Cafeteria
hotel_neighbourhood_df = pd.concat([df_hotel, clean_park_data, df_Restaurant, df_Shopping], ignore_index=True)

hotel_neighbourhood_df = hotel_neighbourhood_df[hotel_neighbourhood_df.categories!='University']
hotel_neighbourhood_df = hotel_neighbourhood_df[hotel_neighbourhood_df.categories!='Office']
hotel_neighbourhood_df = hotel_neighbourhood_df[hotel_neighbourhood_df.categories!='College Residence Hall']
hotel_neighbourhood_df = hotel_neighbourhood_df[hotel_neighbourhood_df.categories!='Building']

hotel_neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,Britannia Hotel Manchester,Hotel,Portland St,53.479056,-2.237239,M1 3LA,Greater Manchester
1,NCP Major Street Car Park,Parking,Major St,53.479008,-2.235512,M1,Greater Manchester
2,"Q-Park Piazza Car Park, Manchester",Parking,75 St James Street,53.477661,-2.241282,M1 4BP,Greater Manchester
3,"Q-Park Deansgate North Car Park, Manchester",Parking,2 Chapel Street,53.485912,-2.245509,M3 7WJ,Greater Manchester
4,"Q-Park Piccadilly Place Car Park, Manchester",Parking,Whitworth Street,53.475538,-2.239354,M1 3BN,Greater Manchester
5,Manchester Fort Retail Park,Shopping Plaza,Cheetham Hill Road,53.497997,-2.235790,M8 8EP,Greater Manchester
6,NCP Aquatic Cntr. Car Park,Parking,Booth St. East,53.465915,-2.232556,M13 9SS,Lancashire
10,The Restaurant Bar and Grill,Steakhouse,14 John Dalton Street,53.480527,-2.247170,M2 6JR,Greater Manchester
11,Gino D'Acampo - My Restaurant,Italian Restaurant,98 Corporation St,53.484844,-2.242675,M4 3TR,Greater Manchester
12,Tae's Tavern - Caribbean Restaurant Bar Cafe,Caribbean Restaurant,187 - 189 Chapel Street,53.483456,-2.255137,M3 5EQ,Greater Manchester


In [20]:
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=15)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map